In [1]:
import os
import numpy as np
import spacy
import gensim
import collections
import smart_open
import random




In [2]:
nlp = spacy.load('en_core_web_lg')

In [3]:
query = "Fetch my messages related to sports"

query = nlp(query)

entity = []

for token in query:
    if token.pos_ == 'PROPN' or token.pos_ == 'NOUN':
        entity.append(token.text.lower())


In [5]:
print(entity)

['messages', 'sports']


In [14]:
# Set file names for train and test data
#test_data_dir = '{}'.format(os.sep).join([gensim.__path__[0], 'test', 'test_data'])
test_data_dir = '/home/unity/IVR/ConversationalIvr/utility/unity_message'
lee_train_file = test_data_dir + os.sep + 'train_question.cor'
lee_test_file = test_data_dir + os.sep + 'test_question.cor'
train_answer_file = test_data_dir + os.sep + 'train_answer.cor'
vocab_file = '/home/unity/IVR/ConversationalIvr/utility' + os.sep + 'text8'



def read_corpus(fname, tokens_only=False):
    with smart_open.smart_open(fname, encoding="iso-8859-1") as f:
        for i, line in enumerate(f):
            if tokens_only:
                yield gensim.utils.simple_preprocess(line)
            else:
                # For training data, add tags
                yield gensim.models.doc2vec.TaggedDocument(gensim.utils.simple_preprocess(line), [i])

def train_model():
    model = gensim.models.doc2vec.Doc2Vec(vector_size=1024, min_count=1, epochs=10000, window=2, dbow_words=1, dm=1)
    model.build_vocab(train_corpus)
    model.train(train_corpus, total_examples=model.corpus_count, epochs=model.epochs)
    model.save("doc2vec_unity_message")
    return model

def load_model():
    model_name = '/home/unity/IVR/ConversationalIvr/utility/unity_message/doc2vec_unity_message'
    model = gensim.models.doc2vec.Doc2Vec.load(model_name)
    return model

In [55]:
def test_model(query):
    # Load the Pre-Trained Model on Question Answer Data Set #
    print ('Load the Model')
    model = load_model()
    ranks = []
    second_ranks = []
    query = model.infer_vector(query.split(" "))
    #doc_id = random.randint(0, len(train_corpus) - 1)
    #inferred_vector = model.infer_vector(train_corpus[doc_id].words)
    sims = model.docvecs.most_similar([query], topn=len(model.docvecs))
    print (sims)

    print('Question ({}): «{}»\n'.format(sims[0][0], ' '.join(train_corpus[sims[0][0]].words)))
    print('Answer  ({}): «{}»\n'.format(sims[0][0], ' '.join(train_answer_corpus[sims[0][0]].words)))
    print("Question Score",sims[0][1])
    if sims[0][1] > 0.7:        
        answer = ' '.join(train_answer_corpus[sims[0][0]].words)
        return answer
    else:
        answer = "Sorry i am not very sure what to answer for this query"
        return answer

print("Reading Corpus for doc2vec")
train_corpus = list(read_corpus(lee_train_file))
train_answer_corpus = list(read_corpus(train_answer_file))
print("Reading corpus completed")

model = train_model()

Reading Corpus for doc2vec
Reading corpus completed


In [61]:
query = "Speech to text not for web inbox?"
test_model (query)

Load the Model
[(9, 0.6435563564300537), (10, 0.44871705770492554), (3, 0.4264877438545227), (4, 0.3911691904067993), (8, 0.36687171459198), (1, 0.3431747257709503), (2, 0.2703357934951782), (5, 0.20731395483016968), (0, 0.18818405270576477), (7, 0.04478814825415611), (6, -0.05610649660229683)]
Question (9): «speech to text not coming on the web inbox»

Answer  (9): «two mazor reasons are there one is hold until transcription recieved check can be there and second is certificat expiration»

Question Score 0.6435563564300537


/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


'Sorry i am not very sure what to answer for this query'